In [ ]:
import os
import sys 
sys.path.append("..")
from i2e import *
from PIL import Image
from tqdm import tqdm
from torchvision import datasets
from torchvision.transforms import v2

In [ ]:
dataset_path = '/ssd/Datasets/CIFAR10/'
root_out = '/ssd/Datasets/I2E_CIFAR10/'
batch_size = 128
workers = 28

In [ ]:
net = I2E(ratio=0.07)
net.eval()
net.set_weight()

In [ ]:
class I2E_CIFAR10(datasets.CIFAR10):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False, root_out=root_out):
        super(I2E_CIFAR10, self).__init__(root, train, transform, target_transform, download)
        self.root_out = root_out + 'train/' if train else root_out + 'val/'

    def __getitem__(self, index):

        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        img_dvs = net(img.unsqueeze(0)).squeeze()
        save_path = f'{self.root_out}{target}/'
        if not os.path.exists(save_path):
            os.makedirs(save_path, exist_ok=True)
        np.savez_compressed(f'{save_path}{index}', img_dvs.numpy().astype(bool))

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img_dvs, target

In [ ]:
transform_train = v2.Compose([
    v2.PILToTensor(),
    v2.Resize((128, 128)),
    v2.ToDtype(torch.float32, scale=True),
])

transform_test = v2.Compose([
    v2.PILToTensor(),
    v2.Resize((128, 128)),
    v2.ToDtype(torch.float32, scale=True),
])

train_dataset = I2E_CIFAR10(root=dataset_path, train=True, download=True, transform=transform_train, root_out=root_out)
val_dataset = I2E_CIFAR10(root=dataset_path, train=False, download=True, transform=transform_test, root_out=root_out)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)

In [ ]:
for i, (images, target) in enumerate(tqdm(train_loader)):
    pass

In [ ]:
for i, (images, target) in enumerate(tqdm(val_loader)):
    pass